In [35]:
import requests
import os
import bs4
import re
import datetime
import csv
import pandas as pd

I am setting up this script to only pull filings posted since last run 

In [36]:
LAST_RUN = datetime.datetime.strptime('01/01/2018', "%m/%d/%Y").date()
FORM = 'DEF 14A'
BASE_URL = "https://www.sec.gov/cgi-bin/srch-edgar"
DF_COLS = ['company', 'filing_year', 'median_salary', 'ceo_salary']
sal_df = pd.DataFrame(columns=DF_COLS)

### HELPER FUNCTIONS

In [37]:
def process_pay_section(pay_section):
    """ Remove Symbols to make it 
        easier to parse for salaries"""
    pay_section = pay_section.strip("\n")
    # Subst mult white space with single
    pay_section = ' '.join(pay_section.split())
    # Get rid of multiple spaces after dollar signs
    pay_section = pay_section.replace('$ ', '$')
    pay_section = pay_section.replace(' : ', ':')

    symbs = ["&", "#", "&nbsp;", "nbsp", "\n", ';']
    for s in symbs:
        pay_section = pay_section.replace(s, "")
    return pay_section

In [57]:
def get_ceo_mult_from_ratios(pay_section):
    """ Gets the CEO Multiplier from pay ratio between median employee salary
        and CEO salary
    """
    ratios = re.findall('\d+[,d+]?\.?\d*(?::|[\s-]to[\s-])1', pay_section)
    ratios = [r.replace(',', '') for r in ratios]
    if ratios == []:
        ratios = re.findall('[(]?\d+\.?\d*[)]? times that', pay_section)

    if ratios != []:
        ratio_str = ratios[0]
        if ':' in ratio_str:
            ceo_mult = float(ratio_str[0:ratio_str.find(':')])
        elif 'to' in ratio_str:
            to_find = '-to-' if '-' in ratio_str else ' to '
            ceo_mult = float(ratio_str[0:ratio_str.find(to_find)])
        elif 'times that' in ratio_str:
            ratio_str = ratio_str.replace('(', '').replace(')', '')
            ceo_mult = float(ratio_str[0:ratio_str.find('times')])
            
        return ceo_mult
    else:
        return None

In [58]:
def get_emp_ceo_sal_from_mult(string_salaries, ceo_mult):
    num_salaries = [float(sal.replace('$', '').replace(',', '')) for sal in string_salaries]
    num_salaries.sort()
     
    # Use CEO Mult to get Employee Salary
    emp_salary = -1
    ceo_salary = -1
    for i in range(len(num_salaries) - 1):
        for j in range(i + 1, len(num_salaries)):
            lower_sal = num_salaries[i]
            higher_sal = num_salaries[j]
            if abs(higher_sal / lower_sal - ceo_mult) < 2:
                ceo_salary = higher_sal
                emp_salary = lower_sal
        
    if len(num_salaries) == 2 and emp_salary == -1 and ceo_salary == -1:
        emp_salary = num_salaries[0]
        ceo_salary = num_salaries[1]
                
    return emp_salary, ceo_salary

In [59]:
def process_single_page(table_of_filings, sal_df):
    num_filings = len(table_of_filings)
    
    # Loop through every entry in table
    for i in range(1, num_filings):
        row = table_of_filings[i]
        cols = row.findAll("td")
        
        comp_name = cols[1].get_text()
        filing_date = datetime.datetime.strptime(cols[4].get_text(), "%m/%d/%Y").date()
    
        # Only grab those filings that have been posted since we last pulled
        if filing_date >= LAST_RUN:
            txt_url = cols[2].find()['href']
            comp_def14a_html = requests.get('https://www.sec.gov/%s' %txt_url).text
            comp_def14a = re.sub('<[^<]+?>', '', comp_def14a_html)
             # Remove TOC
            comp_def14a = comp_def14a[int(0.20 * len(comp_def14a)): ]
        
            # If it doesn't have median or pay ratio section move on
            if comp_def14a.find("median") == -1 or (comp_def14a.find('Pay Ratio') == -1 
                                                    and comp_def14a.find('PAY RATIO') == -1):
                continue
            else:
                pay_ratio_start  = re.search('[CEO]?Pay Ratio[\sDisclosure]?', comp_def14a, re.IGNORECASE).start()           
                # Process Pay Section so I can get what I need
                pay_section = process_pay_section(comp_def14a[pay_ratio_start:pay_ratio_start + 5000])
        
            
                ceo_mult = get_ceo_mult_from_ratios(pay_section)
                if ceo_mult is None:
                    print(comp_name)
                    print('Could not get Ratio')
                    print(pay_section)
                else:
                     # Get All Saries
                    all_salaries = re.findall('\$\d{1,3},\d{3,3},?\d*', pay_section)
                    emp_salary, ceo_salary = get_emp_ceo_sal_from_mult(all_salaries, 
                                                                   ceo_mult)
                    if emp_salary == -1 and ceo_salary == -1:
                        print(comp_name)
                        print('Couldnt find salaries to match ratio')
                        print(ceo_mult)
                        print(all_salaries)
                        print(pay_section)
                    else:
                        sal_df = sal_df.append(pd.DataFrame([[comp_name, filing_date.year, 
                                                         emp_salary, ceo_salary]],
                                                        columns=DF_COLS))
    
    return sal_df
    
    

In [60]:
# LOOP THROUGH ALL PAGES
count = 100
pos = 1
sal_df = pd.DataFrame(columns=DF_COLS)
page = 1
more_pages = True
while page < 50:
    print('Page', page)
    url = "{base_url}?text={form}&start={pos}&count={count}&first=2018&last=2018".format(base_url=BASE_URL,
                                                                                         form=FORM,
                                                                                         pos=pos,
                                                                                         count=count)
    
    list_of_forms = requests.get(url)
    soup = bs4.BeautifulSoup(list_of_forms.text, 'html.parser')
    table_of_filings = (soup.findAll("table")[4]).findAll("tr")
    sal_df = process_single_page(table_of_filings, sal_df)
    # Ex pos is 1 on page 1, will be 101 on page 2 if count is 100
    pos += count
    page += 1

Page 1
Page 2
AGCO CORP /DE
Couldnt find salaries to match ratio
297.0
['$49,623', '$245,000', '$1,800,000']
PAY RATIOOur analysis began by determining that we had approximately 20,26532employees as of a November 30, 2017 determination date. Although permitted by the SEC, we did not use the 5% de minimis rule to exclude or eliminate any employee group. Approximately 56032employees related to acquisitions we completed during 2017 were excluded from the calculation. After identifying the median employee based on our consistently applied compensation measure of actual total cash compensation, we calculated the annual total compensation using the same methodology we use for the CEO as set forth in the 2017 Summary Compensation Table. The median employee8217s total compensation was approximately $49,623. Based on this methodology, we estimate that the ratio of CEO pay to median employee pay is 297:1. 46Table of ContentsTHE FOLLOWING REPORTS OF THE COMPENSATION COMMITTEE AND THE AUDIT COMMIT

Alkermes plc.
Could not get Ratio
PAY RATIO Under the Dodd-Frank Wall Street Reform and Consumer Protection Act and the related SEC rule (the x201CRulex201D), we are required to provide our shareholders with specified disclosure regarding the relationship of our Chief Executive Officerx2019s total compensation to the total compensation of our median employee, referred to as x201Cpay-ratiox201D disclosure.For 2017, the annual total compensation for our Chief Executive Officer, including the value of employer paid health care benefits, as discussed below, was $9,403,046 and for our median employee was $134,639 (also including the value of employer paid health care benefits), resulting in a pay ratio of approximately seventy to one.  The annual total compensation for our Chief Executive Officerin the pay ratio disclosure differs from the annual total compensation amounts reflected in the Summary Compensation Table because we included the value of employer paid health care benefits (estima

Altisource Asset Management Corp
Couldnt find salaries to match ratio
16052.0
['$51,081', '$2,668,257', '$398,077', '$8,011,048', '$675,000160', '$89,844', '$9,173,9691602016160450', '$450,000160160', '$32,682', '$550,000160', '$117,555', '$1,150,2371602016160450', '$328,863', '$250,000160', '$270,887', '$849,7501602016160350', '$450,000', '$7,988,982', '$22,066', '$32,682']
Pay RatioThe below table shows the ratio of the median annual total compensation of all employees of the Company (excluding the Chief Executive Officer) to the annual total compensation of our Chief Executive Officer. In determining the median employee, a listing was prepared of all current employees as of December 31, 2017. To determine the median employee, we included the annual base salary as of December 31, 201732and incentive compensation related to the 201732fiscal year. Once the median employee was identified, for purposes of comparison to the Chief Executive Officer, we then calculated the total cash compen

Page 4
Page 5
APTARGROUP INC
Couldnt find salaries to match ratio
131.0
['$39,366', '$6,376,678', '$51,000,', '$34,000']
PAY RATIO As required by Section953(b) of the Dodd-Frank Wall Street Reform and Consumer Protection Act, Aptar is providing the following disclosure about the relationship of the annual total compensation of our employees to the annual total compensation of Stephan Tanda, our President and CEO. To better understand this disclosure, we think it is important to give context to our operations. As a global organization, approximately 82% of our employees are located outside of the United States, with approximately 7,000 employees located in Europe and 2,600 employees located in Asia and South America. We strive to create a competitive global compensation program in terms of both the position and the geographic location in which the employee is located. As a result, our compensation program varies amongst each local market in order to allow us to provide a competitive com

Page 6
ATRION CORP
Could not get Ratio
Pay Ratio Disclosure Pursuant to Item 402(u) of Regulation S-K under the Securities Exchange Act of 1934, as amended, we are providing the following information about the relationship of the annual total compensation of our employees and the annual total compensation of our Chief Executive Officer, Mr. David Battat: For 2017, our Chief Executive Officer had total compensation of $4,315,721, as reflected in the Summary Compensation Table at page 17 of this proxy statement. We determined that our median employee146s annual total compensation was $38,242 for 2017. As a result, our Chief Executive Officer146s annual total compensation for 2017 was approximately 113 times the annual total compensation of our median employee. We identified our median employee by examining the 2017 total cash compensation for all individuals, excluding our Chief Executive Officer, who were employed by us on December 31, 2017. We included all employees, whether employed o

...

Page 5388
Page 5389
Page 5390
Page 5391
Page 5392
Page 5393
Page 5394
Page 5395
Page 5396
Page 5397
Page 5398
Page 5399
Page 5400
Page 5401
Page 5402
Page 5403
Page 5404
Page 5405
Page 5406
Page 5407
Page 5408
Page 5409
Page 5410
Page 5411
Page 5412
Page 5413
Page 5414
Page 5415
Page 5416
Page 5417
Page 5418
Page 5419
Page 5420
Page 5421
Page 5422
Page 5423
Page 5424
Page 5425
Page 5426
Page 5427
Page 5428
Page 5429
Page 5430
Page 5431
Page 5432
Page 5433
Page 5434
Page 5435
Page 5436
Page 5437
Page 5438
Page 5439
Page 5440
Page 5441
Page 5442
Page 5443
Page 5444
Page 5445
Page 5446
Page 5447
Page 5448
Page 5449
Page 5450
Page 5451
Page 5452
Page 5453
Page 5454
Page 5455
Page 5456
Page 5457
Page 5458
Page 5459
Page 5460
Page 5461
Page 5462
Page 5463
Page 5464
Page 5465
Page 5466
Page 5467
Page 5468
Page 5469
Page 5470
Page 5471
Page 5472
Page 5473
Page 5474
Page 5475
Page 5476
Page 5477
Page 5478
Page 5479
Page 5480
Page 5481
Page 5482
Page 5483
Page 5484
Page 5485
Page 5486
Page 5487


Page 6208
Page 6209
Page 6210
Page 6211
Page 6212
Page 6213
Page 6214
Page 6215
Page 6216
Page 6217
Page 6218
Page 6219
Page 6220
Page 6221
Page 6222
Page 6223
Page 6224
Page 6225
Page 6226
Page 6227
Page 6228
Page 6229
Page 6230
Page 6231
Page 6232
Page 6233
Page 6234
Page 6235
Page 6236
Page 6237
Page 6238
Page 6239
Page 6240
Page 6241
Page 6242
Page 6243
Page 6244
Page 6245
Page 6246
Page 6247
Page 6248
Page 6249
Page 6250
Page 6251
Page 6252
Page 6253
Page 6254
Page 6255
Page 6256
Page 6257
Page 6258
Page 6259
Page 6260
Page 6261
Page 6262
Page 6263
Page 6264
Page 6265
Page 6266
Page 6267
Page 6268
Page 6269
Page 6270
Page 6271
Page 6272
Page 6273
Page 6274
Page 6275
Page 6276
Page 6277
Page 6278
Page 6279
Page 6280
Page 6281
Page 6282
Page 6283
Page 6284
Page 6285
Page 6286
Page 6287
Page 6288
Page 6289
Page 6290
Page 6291
Page 6292
Page 6293
Page 6294
Page 6295
Page 6296
Page 6297
Page 6298
Page 6299
Page 6300
Page 6301
Page 6302
Page 6303
Page 6304
Page 6305
Page 6306
Page 6307


Page 7028
Page 7029
Page 7030
Page 7031
Page 7032
Page 7033
Page 7034
Page 7035
Page 7036
Page 7037
Page 7038
Page 7039
Page 7040
Page 7041
Page 7042
Page 7043
Page 7044
Page 7045
Page 7046
Page 7047
Page 7048
Page 7049
Page 7050
Page 7051
Page 7052
Page 7053
Page 7054
Page 7055
Page 7056
Page 7057
Page 7058
Page 7059
Page 7060
Page 7061
Page 7062
Page 7063
Page 7064
Page 7065
Page 7066
Page 7067
Page 7068
Page 7069
Page 7070
Page 7071
Page 7072
Page 7073
Page 7074
Page 7075
Page 7076
Page 7077
Page 7078
Page 7079
Page 7080
Page 7081
Page 7082
Page 7083
Page 7084
Page 7085
Page 7086
Page 7087
Page 7088
Page 7089
Page 7090
Page 7091
Page 7092
Page 7093
Page 7094
Page 7095
Page 7096
Page 7097
Page 7098
Page 7099
Page 7100
Page 7101
Page 7102
Page 7103
Page 7104
Page 7105
Page 7106
Page 7107
Page 7108
Page 7109
Page 7110
Page 7111
Page 7112
Page 7113
Page 7114
Page 7115
Page 7116
Page 7117
Page 7118
Page 7119
Page 7120
Page 7121
Page 7122
Page 7123
Page 7124
Page 7125
Page 7126
Page 7127


Page 7848
Page 7849
Page 7850
Page 7851
Page 7852
Page 7853
Page 7854
Page 7855
Page 7856
Page 7857
Page 7858
Page 7859
Page 7860
Page 7861
Page 7862
Page 7863
Page 7864
Page 7865
Page 7866
Page 7867
Page 7868
Page 7869
Page 7870
Page 7871
Page 7872
Page 7873
Page 7874
Page 7875
Page 7876
Page 7877
Page 7878
Page 7879
Page 7880
Page 7881
Page 7882
Page 7883
Page 7884
Page 7885
Page 7886
Page 7887
Page 7888
Page 7889
Page 7890
Page 7891
Page 7892
Page 7893
Page 7894
Page 7895
Page 7896
Page 7897
Page 7898
Page 7899
Page 7900
Page 7901
Page 7902
Page 7903
Page 7904
Page 7905
Page 7906
Page 7907
Page 7908
Page 7909
Page 7910
Page 7911
Page 7912
Page 7913
Page 7914
Page 7915
Page 7916
Page 7917
Page 7918
Page 7919
Page 7920
Page 7921
Page 7922
Page 7923
Page 7924
Page 7925
Page 7926
Page 7927
Page 7928
Page 7929
Page 7930
Page 7931
Page 7932
Page 7933
Page 7934
Page 7935
Page 7936
Page 7937
Page 7938
Page 7939
Page 7940
Page 7941
Page 7942
Page 7943
Page 7944
Page 7945
Page 7946
Page 7947


KeyboardInterrupt: 

In [61]:
sal_df.shape

(1017, 4)

In [62]:
sal_df.head()

,company,filing_year,median_salary,ceo_salary
0,1ST CONSTITUTION BANCORP,2018,53278.0,1487675.0
0,"22nd Century Group, Inc.",2018,36608.0,711375.0
0,"2U, Inc.",2018,60703.0,3389499.0
0,3D SYSTEMS CORP,2018,55683.0,1933108.0
0,A. H. Belo Corp,2018,54142.0,2053650.0


In [64]:
# Get Rid of OUtliers
sal_df['ratio'] = sal_df['median_salary'] / sal_df['ceo_salary']

In [70]:
sal_df_final = sal_df.loc[sal_df['ratio'] <= 0.8, :]

In [74]:
del sal_df_final['ratio']

In [75]:
sal_df_final.to_csv('sec_employee_ceo_median_salary.csv', index=False)